# Beginning

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prettytable
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller 
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.tsa.stattools as st
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
import datetime
from pmdarima.arima import auto_arima
import arch as arch
from statsmodels.stats.diagnostic import het_arch
import statsmodels.graphics.tsaplots as sgt
import statsmodels.api as sm

# Import packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from functools import partial
from torch.autograd import Variable
import math
import time

import pandas as pd
import numpy as np
import random

import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

# set the random seeds for deterministic results
SEED = 4601
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = "./"

def readx(pathname):
    
    df = pd.read_csv(path + pathname + ".csv", index_col=[0], header=[0])
    return df

electricity_total_df = readx("Electricity")
electricity_total_df = electricity_total_df[0:595]

In [ ]:
electricity_total_df.columns = ["Thermal Electricity", "New Energy"]
electricity_total_df.plot(grid = True, figsize=(12,5))
plt.xlabel('Date (in months)', fontsize = 12)
plt.ylabel('Electricity net generation (in Mkwh)', fontsize = 12)
plt.legend()
plt.show()
electricity_total_df.columns = ["Electricity_Fire", "Electricity_Nonfire"]

# Data 1：Thermal Electricity

## 1. Stationarity：Log

In [ ]:
electricity_fire_df = electricity_total_df[:588]
electricity_fire_df = pd.DataFrame(electricity_fire_df["Electricity_Fire"].values, index = electricity_fire_df.index, columns = ["EG"])
electricity_fire_array = electricity_fire_df.values.ravel()
electricity_fire_series = pd.Series(electricity_fire_array, index = electricity_fire_df.index)

In [ ]:
def pre_table(table_name, table_rows):
    
    table = prettytable.PrettyTable()  
    table.field_names = table_name  
    for i in table_rows:  
        table.add_row(i)  
    return table


def adf_val(ts, ts_title, acf_title, pacf_title):
    plt.figure(figsize = (10,5))
    plt.plot(ts)  
    plt.title(ts_title)  
    plt.show()

    plot_acf(ts, lags=40, title=acf_title).show()  # ACF
    plot_pacf(ts, lags=40, title=pacf_title).show()  # PACF

    adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts)  
    table_name = ['adf', 'pvalue', 'usedlag', 'nobs', 'critical_values', 'icbest']  
    table_rows = [[adf, pvalue, usedlag, nobs, critical_values, icbest]]  
    adf_table = pre_table(table_name, table_rows)   
    print ('stochastic score')  
    print (adf_table)
    return adf, pvalue, critical_values


adf, pvalue1, critical_values = adf_val(electricity_fire_series, 'Time series', 'ACF', 'PACF')  

In [ ]:
# Take log, with ADF test

def criteria(adf, pvalue):
    return adf < critical_values['5%'] and pvalue < 0.05

def best_log(ts, max_log=5):
        for i in range(1, max_log):
            ts = np.log(ts) 
            adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts)  
            print(pvalue)
            if criteria(adf, pvalue):
                print(adf)
                print ('The best log n is: {0}'.format(i))  
                return i  
                

# For the original datasets
result = criteria(adf, pvalue1)
print(result)
print("-----------------------------------------------------------------------")
log_n = best_log(electricity_fire_series)
print(log_n)

# Total 4 times

# One time log is the best, which is then selected
electricity_fire_series_log = np.log(electricity_fire_series)
adf, pvalue1, critical_values = adf_val(electricity_fire_series_log, 'Time series', 'ACF', 'PACF') 

## 2. Stationarity：Differencing

In [ ]:
# First time differencing

electricity_fire_series_log_diff = electricity_fire_series_log.diff()
# rb_diff.isnull()
electricity_fire_series_log_diff = electricity_fire_series_log_diff.fillna(0)

adf, pvalue1, critical_values = adf_val(electricity_fire_series_log_diff, 'Time series (log diff)', 'ACF', 'PACF')
print(adf, pvalue1)

In [ ]:
# First time ACF and PACF performance well

#ljungbox-test
def test_stochastic(ts, lag):
 p_value = acorr_ljungbox(ts, lags=lag)
 return p_value

test_stochastic(electricity_fire_series_log_diff, 24)

In [ ]:
acorr_ljungbox(electricity_fire_series_log_diff, lags=[24])

## 3. Stationarity：Seasonal Differencing

In [ ]:
# Visualization
decomposition = seasonal_decompose(electricity_fire_series_log_diff, period=12)
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(30, 15)

In [ ]:
# Whether the effect is 24 periods or not

def adf_val(ts, ts_title, acf_title, pacf_title):
    plt.figure(figsize = (10,5))
    plt.plot(ts)  
    plt.title(ts_title)  
    plt.show()

    plot_acf(ts, lags=40, title=acf_title).show()  # ACF
    plot_pacf(ts, lags=40, title=pacf_title).show()  # PACF

    adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts)  
    table_name = ['adf', 'pvalue', 'usedlag', 'nobs', 'critical_values', 'icbest']  
    table_rows = [[adf, pvalue, usedlag, nobs, critical_values, icbest]]  
    adf_table = pre_table(table_name, table_rows)   
    print ('stochastic score')  
    print (adf_table)
    return adf, pvalue, critical_values

In [ ]:
# Seasonal Differencing
electricity_fire_series_log_diff_Season = electricity_fire_series_log_diff.diff(12)
# rb_diff.isnull()
electricity_fire_series_log_diff_Season = electricity_fire_series_log_diff_Season.fillna(0)

adf, pvalue1, critical_values = adf_val(electricity_fire_series_log_diff_Season, 'Time series (log diff season)', 'ACF', 'PACF')
print(adf, pvalue1)

## 4. SARIMA model fitting

In [ ]:
# Trials
ARIMA_model = auto_arima(electricity_fire_series_log, 
                      start_p=1, 
                      start_q=1,
                      test='adf', # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                      d=None,# let model determine 'd'
                      seasonal=False, # No Seasonality for standard ARIMA
                      trace=False, 
                      error_action='warn', #shows errors ('ignore' silences these)
                      suppress_warnings=True,
                      stepwise=True)

ARIMA_model

In [ ]:
print(ARIMA_model.summary())

In [ ]:
ARIMA_model.plot_diagnostics(figsize=(15,12), lags = 40)
plt.show()

 ### from the diagnostics graphs, 
 1. residual: mean zero, but variance seems to be left-tailed
 2. Histogram plus KDE estimate: skewed KDE
 3. Normal QQ: quite a few points are ont on the line -> not normal residual
 4. ACF: a few out -> not accurate enough

### Continue testing seasonality

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(0, 1, 0), seasonal_order = (0,1,0,12))
results_trial = sarima_trial.fit()
results_trial.summary()

#### SAR Direction

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(1, 1, 1), seasonal_order = (3,0,0,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(1, 1, 1), seasonal_order = (2,0,0,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
sarima_smafinal = SARIMAX(electricity_fire_series_log, order=(1, 1, 1), seasonal_order = (2,0,0,12))
results_smafinal = sarima_smafinal.fit()
results_smafinal.summary()

In [ ]:
results_smafinal.plot_diagnostics(lags= 40, figsize=(15,12))
plt.show()

#### SMA Direction

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(1, 1, 0), seasonal_order = (0,1,1,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
sarima_sarfinal = SARIMAX(electricity_fire_series_log, order=(0, 1, 1), seasonal_order = (0,1,1,12))
results_sarfinal = sarima_sarfinal.fit()
results_sarfinal.summary()

In [ ]:
results_sarfinal.plot_diagnostics(lags= 40, figsize=(15,12))
plt.show()

#### Combination

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(0, 1, 0), seasonal_order = (1,0,1,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
sarima_trial = SARIMAX(electricity_fire_series_log, order=(1, 1, 2), seasonal_order = (1,0,1,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
sarima_sarmafinal = SARIMAX(electricity_fire_series_log, order=(1, 1, 1), seasonal_order = (1,0,1,12), method = "ML")
results_sarmafinal = sarima_sarmafinal.fit()
results_sarmafinal.summary()

In [ ]:
results_sarmafinal.plot_diagnostics(lags= 40, figsize=(15,12))
plt.show()

# Data2：New energy

In [ ]:
electricity_nonfire_df = electricity_total_df[:588]
electricity_nonfire_df = pd.DataFrame(electricity_nonfire_df["Electricity_Nonfire"].values, index = electricity_nonfire_df.index, columns = ["EG"])
electricity_nonfire_array = electricity_nonfire_df.values.ravel()
electricity_nonfire_series = pd.Series(electricity_nonfire_array, index = electricity_nonfire_df.index)

In [ ]:
def pre_table(table_name, table_rows):
    
    table = prettytable.PrettyTable()  
    table.field_names = table_name  
    for i in table_rows:  
        table.add_row(i)  
    return table


def adf_val(ts, ts_title, acf_title, pacf_title):
    plt.figure(figsize = (10,5))
    plt.plot(ts)  
    plt.title(ts_title)  
    plt.show()

    plot_acf(ts, lags=40, title=acf_title).show()  # ACF
    plot_pacf(ts, lags=40, title=pacf_title).show()  # PACF

    adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts)  
    table_name = ['adf', 'pvalue', 'usedlag', 'nobs', 'critical_values', 'icbest']  
    table_rows = [[adf, pvalue, usedlag, nobs, critical_values, icbest]]  
    adf_table = pre_table(table_name, table_rows)   
    print ('stochastic score')  
    print (adf_table)
    return adf, pvalue, critical_values


adf, pvalue1, critical_values = adf_val(electricity_nonfire_series, 'Time series', 'ACF', 'PACF')  

In [ ]:
def criteria(adf, pvalue):
    return adf < critical_values['5%'] and pvalue < 0.05

def best_log(ts, max_log=5):
        for i in range(1, max_log):
            ts = np.log(ts) 
            adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts) 
            print(pvalue)
            if criteria(adf, pvalue):
                print(adf)
                print ('The best log n is: {0}'.format(i)) 
                return i  

result = criteria(adf, pvalue1)
print(result)
print("-----------------------------------------------------------------------")
log_n = best_log(electricity_nonfire_series)
print(log_n)
electricity_nonfire_series_log = np.log(electricity_nonfire_series)
adf, pvalue1, critical_values = adf_val(electricity_nonfire_series_log, 'Time series', 'ACF', 'PACF') 

In [ ]:
electricity_nonfire_series_log_diff = electricity_nonfire_series_log.diff()
# rb_diff.isnull()
electricity_nonfire_series_log_diff = electricity_nonfire_series_log_diff.fillna(0)

adf, pvalue1, critical_values = adf_val(electricity_nonfire_series_log_diff, 'Time series (log diff)', 'ACF', 'PACF')
print(adf, pvalue1)

In [ ]:
def test_stochastic(ts, lag):
 p_value = acorr_ljungbox(ts, lags=lag) 
 return p_value

test_stochastic(electricity_nonfire_series_log_diff, 24)
white_noise_garch = acorr_ljungbox(electricity_nonfire_series_log_diff, lags = [24], return_df=True)
white_noise_garch

In [ ]:
decomposition = seasonal_decompose(electricity_nonfire_series_log_diff, period=12)
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(30, 15)

In [ ]:
def adf_val(ts, ts_title, acf_title, pacf_title):
    plt.figure(figsize = (10,5))
    plt.plot(ts)  
    plt.title(ts_title)  
    plt.show()

    plot_acf(ts, lags=40, title=acf_title).show()  # ACF
    plot_pacf(ts, lags=40, title=pacf_title).show()  # PACF

    adf, pvalue, usedlag, nobs, critical_values, icbest = adfuller(ts)  
    table_name = ['adf', 'pvalue', 'usedlag', 'nobs', 'critical_values', 'icbest']  
    table_rows = [[adf, pvalue, usedlag, nobs, critical_values, icbest]]  
    adf_table = pre_table(table_name, table_rows)   
    print ('stochastic score')  
    print (adf_table)
    return adf, pvalue, critical_values

In [ ]:
# Seasonal differencing
electricity_nonfire_series_log_diff_Season = electricity_nonfire_series_log_diff.diff()
# rb_diff.isnull()
electricity_nonfire_series_log_diff_Season = electricity_nonfire_series_log_diff_Season.fillna(0)

adf, pvalue1, critical_values = adf_val(electricity_nonfire_series_log_diff_Season, 'Time series (log diff season)', 'ACF', 'PACF')
print(adf, pvalue1)

In [ ]:
ARIMA_model = auto_arima(electricity_nonfire_series_log, 
                      start_p=1, 
                      start_q=1,
                      test='adf', # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                      d=None,# let model determine 'd'
                      seasonal=False, # No Seasonality for standard ARIMA
                      trace=False, 
                      error_action='warn', #shows errors ('ignore' silences these)
                      suppress_warnings=True,
                      stepwise=True)

ARIMA_model

In [ ]:
print(ARIMA_model.summary())

In [ ]:
ARIMA_model.plot_diagnostics(figsize=(15,12), lags = 40)
plt.show()

In [ ]:
sarima_trial = SARIMAX(electricity_nonfire_series_log, order=(0, 1, 0), seasonal_order = (1,1,1,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
# Try SARIMA
# Hyper parameters：(p, 1, q) * (P, D, Q, 12)
SARIMA_model = auto_arima(electricity_nonfire_series_log, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, 
                         m=12, 
                         start_P=0,
                         max_P=3,
                         start_Q=0,
                         max_Q=3,
                         seasonal=True, #set to seasonal
                         d=1, #order of differencing = 1
                         max_D=3, 
                         trace=False,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)
SARIMA_model

In [ ]:
SARIMA_model.summary()

In [ ]:
sarima_trial = SARIMAX(electricity_nonfire_series_log, order=(1, 1, 1), seasonal_order = (1,0,1,12))
results_trial = sarima_trial.fit()
results_trial.summary()

In [ ]:
results_trial.plot_diagnostics(lags= 40, figsize=(15,12))
plt.show()

# Prediction

### Forecast fire electricity data

In [ ]:
# predicted value (transformed) and confidence interval(transformed)
sarima_trial = SARIMAX(electricity_fire_series_log, order=(1, 1, 1), seasonal_order = (1,0,1,12)).fit(dis=-1)
n_periods = 7
fc = sarima_trial.forecast(steps = n_periods)
print(fc)

getforecast = sarima_trial.get_forecast(steps = n_periods)

confint = getforecast.conf_int()
print(confint)

predicted_data = [12.213143, 12.107795, 12.083265, 11.992581, 12.102263, 12.288561, 12.462170]

In [ ]:
# index predicted values (588-594)
index_of_fc = np.arange(len(electricity_fire_series_log_diff_Season), len(electricity_fire_series_log_diff_Season)+n_periods)
print(index_of_fc)

In [ ]:
fc = [12.215872,12.108351,12.085103,11.993450,12.103748,12.289804,12.464019]
lower_y = [12.138070,12.018075,11.990040,11.895923,12.004613,12.189421,12.362553]
upper_y = [12.293674,12.198627,12.180166,12.090977,12.202883,12.390188,12.565486]

In [ ]:
#original value for true tail(n=7)
electricity_fire_total_df = electricity_total_df["Electricity_Fire"]
electricity_fire_total_df.tail(n=7)

In [ ]:
#log value for true tail(n=7)
electricity_fire_total_df
electricity_fire_total_df_array = electricity_fire_total_df.values.ravel()
index_of_total = np.arange(len(electricity_fire_total_df))
electricity_fire_total_df_series = pd.Series(electricity_fire_total_df_array, index = index_of_total)
electricity_fire_total_df_series_log = np.log(electricity_fire_total_df_series)
electricity_fire_total_df_series_log_tail = electricity_fire_total_df_series_log.tail(n=7)
print(electricity_fire_total_df_series_log_tail)

In [ ]:
#plot the graph for transformed data
fc_series = pd.Series(fc,index=index_of_fc)
lower_series = pd.Series(lower_y, index=index_of_fc)
upper_series = pd.Series(upper_y, index=index_of_fc)
print(fc_series)
print(lower_series)
print(upper_series)

# plot
plt.plot()
plt.plot(fc_series, color='darkgreen')
plt.plot(electricity_fire_total_df_series_log_tail, color='darkblue')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Forecast of Electricity Fire")
plt.show()

#### Forecast the original value

In [ ]:
# original tail(n=7)
electricity_fire_total_df_tail_or = electricity_fire_total_df.tail(n=7)
electricity_fire_total_df_tail_or_array = electricity_fire_total_df_tail_or.values.ravel()
electricity_fire_total_df_tail_or_series = pd.Series(electricity_fire_total_df_tail_or_array, index = index_of_fc)

In [ ]:
# plot the graph for orogonal data (not for use because no CI)
fc_series_or = [202128.6879,181572.7582,177420.7305,161892.5241,180778.3767,217752.757,259200.9759]
fc_series_or_series = pd.Series(fc_series_or,index=index_of_fc)
print(electricity_fire_total_df_tail_or_series)
print(fc_series_or_series)

# plot
plt.plot()
plt.plot(fc_series_or_series, color='darkgreen')
plt.plot(electricity_fire_total_df_tail_or_series, color='darkblue')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Forecast of Electricity Fire (original data)")
plt.show()

### Forecast new energy data

#### Forecast the log transformed value

In [ ]:
# predicted value (transformed) and confidence interval(transformed)
electricity_nonfire_series_log = np.log(electricity_nonfire_series)
sarima_trial = SARIMAX(electricity_nonfire_series_log, order=(1, 1, 1), seasonal_order = (1,0,1,12)).fit(dis=-1)
n_periods = 7
fc = sarima_trial.forecast(steps = n_periods)
print(fc)

getforecast = sarima_trial.get_forecast(steps = n_periods)


In [ ]:
confint = getforecast.summary_frame()
print(confint)

In [ ]:
# index predicted values (588-594)
index_of_fc = np.arange(len(electricity_nonfire_series_log_diff_Season), len(electricity_nonfire_series_log_diff_Season)+n_periods)
print(index_of_fc)

In [ ]:
fc = [11.934303,11.828750,11.883975,11.831995,11.891902,11.887474,11.868540]
lower_y = [11.866696,11.742284,11.787066,11.728529,11.783979,11.776326,11.754928]
upper_y = [12.001911,11.915216,11.980883,11.935462,11.999825,11.998621,11.982151]

In [ ]:
#original value for true tail(n=7)
electricity_nonfire_total_df = electricity_total_df["Electricity_Nonfire"]
electricity_nonfire_total_df.tail(n=7)

In [ ]:
#log value for true tail(n=7)
electricity_nonfire_total_df
electricity_nonfire_total_df_array = electricity_nonfire_total_df.values.ravel()
index_of_total = np.arange(len(electricity_nonfire_total_df))
electricity_nonfire_total_df_series = pd.Series(electricity_nonfire_total_df_array, index = index_of_total)
electricity_nonfire_total_df_series_log = np.log(electricity_nonfire_total_df_series)
electricity_nonfire_total_df_series_log_tail = electricity_nonfire_total_df_series_log.tail(n=7)
print(electricity_nonfire_total_df_series_log_tail)

In [ ]:
#plot the graph for transformed data
fc_series = pd.Series(fc,index=index_of_fc)
lower_series = pd.Series(lower_y, index=index_of_fc)
upper_series = pd.Series(upper_y, index=index_of_fc)
print(fc_series)
print(lower_series)
print(upper_series)

# plot
plt.plot()
plt.plot(fc_series, color='darkgreen')
plt.plot(electricity_nonfire_total_df_series_log_tail, color='darkblue')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Forecast of New energy Fire")
plt.show()

In [ ]:
# original tail(n=7)
electricity_nonfire_total_df_tail_or = electricity_nonfire_total_df.tail(n=7)
electricity_nonfire_total_df_tail_or_array = electricity_nonfire_total_df_tail_or.values.ravel()
electricity_nonfire_total_df_tail_or_series = pd.Series(electricity_nonfire_total_df_tail_or_array, index = index_of_fc)
print(electricity_nonfire_total_df_tail_or_series)

In [ ]:
# plot the graph for original data (use this)
fc_series_or = [152496.6509,137272.4725,145102.7365,137776.537,146300.4925,145667.496,142945.6774]
fc_series_or_series = pd.Series(fc_series_or,index=index_of_fc)
lower_series_or = [142442.8293,125779.2921,131539.9649,124061.0423,131134.5271,130134.785,127379.7422]
lower_series_or_series = pd.Series(lower_series_or,index=index_of_fc)
upper_series_or = [163066.1132,149524.5691,159672.9596,152582.6965,162726.3118,162530.5072,159875.5533]
upper_series_or_series = pd.Series(upper_series_or,index=index_of_fc)
print(electricity_nonfire_total_df_tail_or_series)
print(fc_series_or_series)
print(lower_series_or_series)
print(upper_series_or_series)

# plot
plt.plot()
plt.plot(fc_series_or_series, color='darkgreen')
plt.plot(electricity_nonfire_total_df_tail_or_series, color='darkblue')
plt.fill_between(lower_series_or_series.index, 
                 lower_series_or_series, 
                 upper_series_or_series, 
                 color='k', alpha=.15)

plt.title("Forecast of New energy Fire (original data)")
plt.show()

# LSTM

## LSTM 1

In [ ]:
# Process data
fireList = np.array(electricity_total_df['Electricity_Fire']).tolist() 
max_value = np.max(fireList)
min_value = np.min(fireList)
scalar = max_value - min_value
fireList = list(map(lambda x: x / scalar, fireList))

# Prepare dataset
LOOKBACK = 2
def create_dataset(dataset, look_back=LOOKBACK):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

data_X, data_Y = create_dataset(fireList)
train_size = 420
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = data_Y[:train_size]
test_X = data_X[train_size:]
test_Y = data_Y[train_size:]
train_X = train_X.reshape(-1, 1, 2)
train_Y = train_Y.reshape(-1, 1, 1)
test_X = test_X.reshape(-1, 1, 2)
train_x = torch.from_numpy(train_X).to(device)
train_y = torch.from_numpy(train_Y).to(device)
test_x = torch.from_numpy(test_X).to(device)

# Define model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(LSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size) 
        
    def forward(self, x):
        x, _ = self.rnn(x) # (seq, batch, hidden)

        s, b, h = x.shape
        x = x.view(s*b, h) # 转换成线性层的输入格式
        x = self.linear(x)
        x = x.view(s, b, -1)
        return x

model = LSTM(2, 4).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Train
for e in range(3000):
    var_x = Variable(train_x).to(torch.float32)
    var_y = Variable(train_y).to(torch.float32)

    out = model(var_x)
    loss = criterion(out, var_y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (e + 1) % 100 == 0: 
        print('Epoch: {}, Loss: {:.5f}'.format(e + 1, loss.data))

# Plot result
fig = plt.figure(figsize=(18,10))
x = [i for i in range(2,595)]
plt.xlabel('Date (in months)', fontsize = 12)  
plt.plot(x, pred_test,color = 'r',alpha = 0.7,label='Prediction') 
plt.plot(x, real,color = 'cornflowerblue',alpha = 0.7,label='Real')                    
plt.legend(loc='best')
plt.title('LSTM Prediction')
plt.savefig('lstm_full_real.png', dpi=400, bbox_inches='tight')  


## LSTM-based Encoder Decoder with Attention Mechanism

In [ ]:
# Prepare train set
trainX = []
trainY = []
for i in range(0,468,12):
    x = fireList[i:i+24]
    y = fireList[i+24:i+36]
    trainX.append(x)
    trainY.append(y)
trainX = np.array(trainX).reshape(-1, 1, 24)
trainY = np.array(trainY).reshape(-1, 1, 12)
trainX = torch.from_numpy(trainX).to(device).to(torch.float32)
trainY = torch.from_numpy(trainY).to(device).to(torch.float32)

# Prepare validation set
valiX = []
valiY = []
for i in range(480,528):
    x = fireList[i:i+24]
    y = fireList[i+24:i+36]
    i += 1
    valiX.append(x)
    valiY.append(y)
valiX = np.array(valiX).reshape(-1, 1, 24)
valiY = np.array(valiY).reshape(-1, 1, 12)
valiX = torch.from_numpy(valiX).to(device).to(torch.float32)
valiY = torch.from_numpy(valiY).to(device).to(torch.float32)

# Define the encoder class applying LSTM
class EncoderLSTM(nn.Module):
    def __init__(self, input_size = 1, hid_dim = 4, n_layers = 1, dropout_p = 0):
        super(EncoderLSTM, self).__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(input_size, hid_dim, n_layers, dropout = dropout_p)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        x = x.view(24,1,-1)
        outputs, (hidden, cell) = self.rnn(x)
        
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n_layers * n directions, batch size, hid_dim]
        # cell = [n_layers * n directions, batch size, hid_dim]
        return outputs, hidden, cell # encoded context vector

# Define the encoder class applying LSTM with attention mechanism
# Note: Here I apply the n_layer LSTM. And n_layer is the same in encoder. 
class DecoderAttLSTM(nn.Module):
    def __init__(self, input_size, hid_dim, output_size, n_layers = 1, dropout_p = 0 , max_length = 12):
        super(DecoderAttLSTM, self).__init__()

        self.input_size = input_size
        self.hid_dim = hid_dim
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.dropout = nn.Dropout(dropout_p)
        self.leakyrelu = nn.LeakyReLU(0.1)
        self.l2 = nn.Linear(hid_dim, 1)
        self.l3 = nn.Linear(hid_dim + input_size, input_size)
        self.rnn = nn.LSTM(input_size, hid_dim, n_layers, dropout = dropout_p)

        self.linear_out = nn.Linear(hid_dim, output_size)


    def forward(self, prev_output, hidden, cell, enc_outputs):

        l1 = nn.Linear(self.n_layers * self.hid_dim, 24).to(device)
        # unsqueeze the output of the previous time-step
        prev_output = prev_output.unsqueeze(0)

        # Attention
        # Q
        q = self.leakyrelu(l1(hidden.view(-1).squeeze(0)))

        # K
        k = self.leakyrelu(self.l2(enc_outputs.squeeze(1)).view(-1))

        # Attention_Weight = Q * K / sqrt(dim_K)
        att_weight = F.softmax(torch.mul(q,k) / math.sqrt(24))

        # context vector = Attention_Weight * V
        series_vec = torch.matmul(att_weight,enc_outputs.squeeze(1))

        input = self.l3(torch.cat((series_vec,prev_output))).unsqueeze(0).unsqueeze(0)

        output, (hidden, cell) = self.rnn(input, (hidden, cell))
        
        return output, hidden, cell

# Define class of out model containing the LSTM based encoder and LSTM decoder with Attention Machanism
class attentionModel(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.linear = nn.Linear(4,1)

    def forward(self, x, y, teacher_forcing_ratio = 0.5):
        # x = [1 24]
        # y = [1 12]
        outputs = torch.zeros(12,1).to(self.device)

        enc_outputs, hidden, cell = self.encoder(x)
        # prepare the last data in x as the first input to decoder
        prev_output = x[0][-1].to(self.device)

        # generate the prediction with decoder
        for t in range(12):
            prev_output, hidden, cell = self.decoder(prev_output, hidden, cell, enc_outputs)
            #print('prev_output: ', prev_output.shape) [1, 1, 4]
            prediction = self.linear(prev_output).squeeze(0).squeeze(0)
            outputs[t] = prediction

        # Apply teacher force with probability teacher_force_ratio
            teacher_force = random.random() < teacher_forcing_ratio
            if teacher_force:
                prev_output = y[0][t]
            else:
                prev_output = prediction.squeeze(0)
        return outputs

# instantiate the encoder, decoder, and model
encoder = EncoderLSTM()
decoder = DecoderAttLSTM(input_size = 1, hid_dim = 4, output_size = 12, n_layers = 2)
model = attentionModel(encoder, decoder, device).to(device)


# initialize all weights from a uniform distribution between -0.08 and +0.08
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Train
def train(model, trainX, trainY, optimizer, criterion, clip):
    model.train()

    epoch_loss = 0
    
    for i in range(len(trainX)):

        x = torch.tensor(trainX[i]).to(device)
        y = torch.tensor(trainY[i]).to(device)
        
        
        optimizer.zero_grad()

        output = model(x,y)

        y = y.view(-1)
        
        loss = criterion(output, y)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()

    return epoch_loss

N_EPOCHS = 300
CLIP = 1
train_loss_record = []
best_valid_loss = float('inf')
progress = ProgressBar()
# start training
for epoch in progress(range(0, N_EPOCHS)):

    #print("Epoch: ", epoch, '\t', time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))

    start_time = time.time()
    
    train_loss = train(model, trainX, trainY, optimizer, criterion, CLIP)
    #valid_loss = validate(model, valiX, valiY, criterion)
    train_loss_record.append(train_loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    '''
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'valid_model.pt')
    '''
    if (epoch + 1) % 10 == 0:
        torch.save(model, f'model{epoch}.pt')
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f}')
        fig = plt.figure(figsize=(8,6))
        x = [i for i in range(len(train_loss_record))]
        plt.plot(x,train_loss_record,label='Loss') 
        plt.show()
        
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

'''
var_x = Variable(dataX)
var_y = Variable(dataY)
pred_test = model(var_x, var_y)
'''

# Forecasting
pred = []

for i in range(47):
    x = torch.tensor(dataX[i]).to(device)
    y = torch.tensor(dataY[i]).to(device)
    outputs = torch.zeros(12,1).to(device)
    #output = model(x,y)
    enc_outputs, hidden, cell = encoder(x)
    prev_output = x[0][-1].to(device)
    for t in range(12):
        prev_output, hidden, cell = decoder(prev_output, hidden, cell, enc_outputs)
        prediction = model.linear(prev_output).squeeze(0).squeeze(0)
        outputs[t] = prediction
        prev_output = prediction.squeeze(0)
    pred += outputs.squeeze(1).tolist()


# Plot results
fig = plt.figure(figsize=(18,10))
x = [i for i in range(len(pred))]
plt.xlabel('Date (in months)', fontsize = 12)  
plt.plot(x,pred,color = 'r',alpha = 0.7,label='Prediction') 
plt.plot(x,real[:564],color = 'cornflowerblue',alpha = 0.7,label='Real')                    
plt.legend(loc='best')
plt.title('Prediction')    
plt.savefig('lstm_full_real.png', dpi=400, bbox_inches='tight')  

# ARCH-type model

In [ ]:
results_sarmafinal.plot_diagnostics(lags= 40, figsize=(15,12))
plt.show()

In [ ]:
electricity_fire_series_prediction = results_sarmafinal.predict()
electricity_fire_array_sarimaresidual = electricity_fire_series_log.values - electricity_fire_series_prediction.values
electricity_fire_array_sarimaresidual[0] = 0
electricity_fire_series_sarimaresidual = pd.Series(electricity_fire_array_sarimaresidual, index = electricity_fire_series_log.index)


In [ ]:
electricity_fire_series_sarimaresidual=results_sarmafinal.resid
electricity_fire_series_sarimaresidual[0] = 0

In [ ]:
plt.figure(figsize = (15, 8))
plt.title(label = "Residual Plot (Thermal Power)")

electricity_fire_series_sarimaresidual.plot()
plt.show()

In [ ]:
LM_pvalue = het_arch(electricity_fire_series_sarimaresidual, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
garch_model = arch.arch_model(electricity_fire_series_sarimaresidual, vol = "GARCH", p=1, q=0)
garch_fitted = garch_model.fit()

In [ ]:
garch_fitted.summary()

In [ ]:
garch_fitted.plot(annualize = "M")

In [ ]:
electricity_nonfire_series_prediction = results_trial.predict()
electricity_nonfire_array_sarimaresidual = electricity_nonfire_series_log.values - electricity_nonfire_series_prediction.values
electricity_nonfire_array_sarimaresidual[0] = 0
electricity_nonfire_series_sarimaresidual = pd.Series(electricity_nonfire_array_sarimaresidual, index = electricity_fire_series_log.index)


In [ ]:
plt.figure(figsize = (15, 8))
plt.title(label = "Residual Plot (New energy)")

electricity_nonfire_series_sarimaresidual.plot()
plt.show()

In [ ]:
LM_pvalue = het_arch(electricity_nonfire_series_sarimaresidual, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
garch_model = arch.arch_model(electricity_nonfire_series_sarimaresidual, vol = "EGARCH", p=1, o=0, q=1)
garch_fitted = garch_model.fit(update_freq = 7)

In [ ]:
garch_fitted.summary()

In [ ]:
garch_fitted.plot(annualize = "M")

In [ ]:
garch_std_resid = pd.Series(garch_fitted.resid / garch_fitted.conditional_volatility)
fig = plt.figure(figsize = (15, 8))

# Residual
garch_std_resid.plot(ax = fig.add_subplot(3,1,1), title = 'GARCH Standardized-Residual', legend = False)

# ACF/PACF
sgt.plot_acf(garch_std_resid, lags = 40, ax=fig.add_subplot(3,2,3))
sgt.plot_pacf(garch_std_resid,lags = 40, ax=fig.add_subplot(3,2,4))

# QQ-Plot & Norm-Dist
sm.qqplot(garch_std_resid, line='s', ax=fig.add_subplot(3,2,5)) 
plt.title("QQ Plot")
fig.add_subplot(3,2,6).hist(garch_std_resid, bins = 40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

In [ ]:
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [24], return_df=True)
white_noise_garch

In [ ]:
sim_forecasts = garch_fitted.forecast(horizon=7, method='simulation')
sim_paths = sim_forecasts.simulations.residual_variances[-1].T
sim = sim_forecasts.simulations

bs_forecasts = garch_fitted.forecast(horizon=7, method='bootstrap')
bs_paths = bs_forecasts.simulations.residual_variances[-1].T
bs = bs_forecasts.simulations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13,5))

x = np.arange(1, 8)

# Plot the paths and the mean, set the axis to have the same limit
axes[0].plot(x, np.sqrt(sim_paths), color='tomato', alpha=0.2)
axes[0].plot(x, np.sqrt(sim_forecasts.residual_variance.iloc[-1]),
    color='k', alpha=1)

axes[0].set_title('Model-based Simulation')
axes[0].set_xticks(np.arange(1, 8))
axes[0].set_xlim(1, 7)

axes[1].plot(x, np.sqrt(bs_paths), color='deepskyblue', alpha=0.2)
axes[1].plot(x,np.sqrt(bs_forecasts.residual_variance.iloc[-1]),
    color='k', alpha=1)

axes[1].set_xticks(np.arange(1, 8))
axes[1].set_xlim(1, 7)

axes[1].set_title('Bootstrap Scenario')
plt.show()

In [ ]:
np.sqrt(sim_forecasts.residual_variance.iloc[-1])

In [ ]:
np.sqrt(bs_forecasts.residual_variance.iloc[-1])